# Delta Ingestion

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/lakelogic/LakeLogic/blob/main/examples/03_data_sources/files/delta/delta_demo.ipynb) 
[![GitHub Repo](https://img.shields.io/badge/GitHub-Repo-blue?logo=github)](https://github.com/lakelogic/LakeLogic/blob/main/examples/03_data_sources/files/delta/delta_demo.ipynb)

## Business Scenario

Delta tables are common in lakehouses. You need validation and rule enforcement before downstream use.

## Value Proposition

- Validate Delta inputs
- Apply quality rules
- Standardize outputs

---

## Goals

1. Read from Delta
2. Validate using contract
3. Review results


## Step 1: Install Dependencies

Install the required extras before running this example.


In [ ]:
from pathlib import Path
from lakelogic import DataProcessor

BASE = Path.cwd()
contract_path = BASE / "delta_contract.yaml"
if not contract_path.exists():
    candidate = BASE / "examples" / "03_data_sources" / "files" / "delta" / "delta_contract.yaml"
    if candidate.exists():
        BASE = candidate.parent
        contract_path = candidate

data_dir = BASE / "data"
data_dir.mkdir(parents=True, exist_ok=True)

delta_path = data_dir / "delta_table"

try:
    import deltalake  # noqa: F401
    HAS_DELTA = True
except Exception:
    HAS_DELTA = False

if not HAS_DELTA:
    print("Delta support not installed. Install the LakeLogic delta extra to run this demo.")
else:
    seed_contract = {
        "version": "1.0.0",
        "dataset": "seed_delta",
        "materialization": {
            "strategy": "overwrite",
            "target_path": str(delta_path),
            "format": "delta",
        },
        "server": {"format": "delta"},
    }
    seed_rows = [
        {"id": 1, "email": "alice@example.com"},
        {"id": 2, "email": ""},
        {"id": 3, "email": "bob@example.com"},
    ]
    try:
        DataProcessor(contract=seed_contract).run(seed_rows, materialize=True)
    except Exception as exc:
        print(f"Unable to create sample delta table: {exc}")

    if not delta_path.exists():
        print(f"Delta table not available: {delta_path.resolve()}")
    else:
        processor = DataProcessor(contract=contract_path)
        result = processor.run_source(delta_path)
        print(result)
        print(f"Good: {len(result.good)} | Bad: {len(result.bad)}")


## Step 2: Review the Contract

Open `delta_contract.yaml` to see the schema, source config, and quality rules.


## Step 3: Run the Contract

Run LakeLogic using the YAML contract. Update credentials as needed.
